# The Problem

We will create a model based on simple artifial neural networks (ANNs) each hand image to the corresponding number the hand is depicting ($0$ to $5$).

# Data Ingest

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import os


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [2]:
def construct_dataset_file(directory):
    df = pd.DataFrame(columns=['filename', 'image'])
    for filename in os.listdir(directory):
        file_location = os.path.join(directory, filename)
        image = tf.keras.utils.load_img(file_location, color_mode='grayscale')
        pixels = np.array(image)
        df.loc[len(df.index)] = [file_location, pixels]
        
    return df

def load_train_df(forced=False):
    TRAIN_IMAGES_DIRECTORY = '/kaggle/input/fingers/train'
    TRAIN_DATASET_FILE = '/kaggle/working/train.csv'

    if not os.path.exists(TRAIN_DATASET_FILE) or forced:
        print("Constructing train dataset")
        df = construct_dataset_file(TRAIN_IMAGES_DIRECTORY)
        df.to_csv(TRAIN_DATASET_FILE)
        return df
    else:
        print("Loading cached train dataset")
        return pd.read_csv(TRAIN_DATASET_FILE)

def load_test_df(forced=False):
    TEST_IMAGES_DIRECTORY = '/kaggle/input/fingers/test'
    TEST_DATASET_FILE = '/kaggle/working/test.csv'

    if not os.path.exists(TEST_DATASET_FILE) or forced:
        print("Constructing test dataset")
        df = construct_dataset_file(TEST_IMAGES_DIRECTORY)
        df.to_csv(TEST_DATASET_FILE)
        return df
    else:
        print("Loading cached test dataset")
        return pd.read_csv(TEST_DATASET_FILE)
    
raw_train_df, raw_test_df = load_train_df(), load_test_df()

Constructing train dataset
Constructing test dataset


In [3]:
raw_train_df.shape, raw_test_df.shape
raw_train_df

,filename,image
0,/kaggle/input/fingers/train/7194c4e9-8e19-496a...,"[[58, 56, 54, 54, 54, 53, 49, 48, 51, 53, 55, ..."
1,/kaggle/input/fingers/train/6c9cec85-6a2f-4c6c...,"[[52, 50, 47, 44, 41, 41, 43, 43, 41, 39, 37, ..."
2,/kaggle/input/fingers/train/12852b4c-bb12-4d1e...,"[[37, 35, 33, 33, 36, 36, 33, 34, 40, 41, 39, ..."
3,/kaggle/input/fingers/train/6dec2dfe-79ce-4e3e...,"[[51, 52, 53, 53, 52, 50, 49, 50, 54, 55, 53, ..."
4,/kaggle/input/fingers/train/bbbe289b-d76f-4157...,"[[67, 66, 65, 66, 68, 70, 72, 72, 71, 68, 63, ..."
...,...,...
17995,/kaggle/input/fingers/train/4df1e9d5-6263-4706...,"[[57, 57, 56, 54, 49, 51, 58, 60, 56, 54, 56, ..."
17996,/kaggle/input/fingers/train/d7d70524-075e-4332...,"[[41, 42, 43, 42, 38, 37, 39, 42, 45, 47, 48, ..."
17997,/kaggle/input/fingers/train/290be607-05c0-43b4...,"[[65, 65, 64, 65, 66, 67, 66, 63, 60, 58, 58, ..."
17998,/kaggle/input/fingers/train/9f327db3-e560-483d...,"[[38, 38, 40, 38, 34, 35, 40, 41, 37, 36, 36, ..."
